In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import roc_auc_score
from sklearn.pipeline import make_pipeline
from category_encoders import OrdinalEncoder

from sklearn.model_selection import cross_val_score

from sklearn.metrics import mean_absolute_error

import pickle
import joblib

In [5]:
df =pd.read_csv("df_inner_join.csv", index_col=0)
df

,name,statff_cnt,cost_per_area,nums_of_franchisees,new_nums_of_franchisee,avr_sales,avr_sales_per_area,start_cost,category,year,debt_ratio,profit,fairness,growth,stability
0,이디야커피,424,209.0,2875,305,18704.0,657.0,12913.0,커피,21,88,4,3,3,3
1,비비큐,233,203.0,1746,270,59456.0,4388.0,9078.0,치킨,28,58,3,3,4,3
2,비에이치씨(BHC),167,225.0,1619,231,52103.0,2851.0,8543.0,치킨,19,205,5,3,4,4
3,배스킨라빈스,1816,390.0,1466,85,57170.0,3060.0,32335.0,아이스크림/빙수,37,24,4,3,3,2
4,투다리,16,298.0,1434,145,11251.0,790.0,4533.0,주점,35,126,5,3,4,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2549,다코기,1,165.0,6,0,31174.0,1743.0,8132.0,치킨,8,99,5,3,2,3
2550,밀크밥버거,65,220.0,6,0,2657.0,194.0,3630.0,분식,17,1292,2,3,3,5
2551,전현주식당,2,220.0,6,6,44344.0,1239.0,8836.0,한식,3,141,4,3,0,4
2552,육개장터,3,220.0,6,0,6168.0,394.0,5450.0,한식,6,-176,1,3,1,1


In [11]:
import numpy
numpy.mean(df['avr_sales'])

26523.00480769231

In [6]:
# 학습 / 평가 데이터셋을 분리합니다.
train, test = train_test_split(df, test_size=0.2, random_state=42)
train, val = train_test_split(train, test_size=0.2, random_state=42)


# feature과 label을 분리합니다.

target = "avr_sales"
features = ["statff_cnt", "cost_per_area", "nums_of_franchisees", "new_nums_of_franchisee", "start_cost", "category", "year", "debt_ratio"]

X_train, y_train = train[features], train[target]
X_test, y_test = test[features], test[target]
X_val, y_val = val[features], val[target]
     

In [42]:
from category_encoders import OrdinalEncoder
from xgboost import XGBRegressor
from sklearn.pipeline import make_pipeline

pipe = make_pipeline(
    OrdinalEncoder(),
    XGBRegressor(
        objective="reg:squarederror", #objective="reg:squarederror"
        eval_metric="rmse",  #eval_metric="rmse"
        n_estimators=200,
        random_state=42,
        n_jobs=-1,
        max_depth=7,
       
        
    ),
)
print(pipe)



Pipeline(steps=[('ordinalencoder', OrdinalEncoder()),
                ('xgbregressor',
                 XGBRegressor(base_score=None, booster=None,
                              colsample_bylevel=None, colsample_bynode=None,
                              colsample_bytree=None, enable_categorical=False,
                              eval_metric='rmse', gamma=None, gpu_id=None,
                              importance_type=None,
                              interaction_constraints=None, learning_rate=None,
                              max_delta_step=None, max_depth=7,
                              min_child_weight=None, missing=nan,
                              monotone_constraints=None, n_estimators=200,
                              n_jobs=-1, num_parallel_tree=None, predictor=None,
                              random_state=42, reg_alpha=None, reg_lambda=None,
                              scale_pos_weight=None, subsample=None,
                              tree_method=None, valida

In [48]:
#하이퍼 파라미터 설정
from scipy.stats.distributions import uniform

params = {
    "xgbregressor__max_depth": [2,4,6,8],
    "xgbregressor__min_child_weight": [1, 3,5,7 , 10],
    "xgbregressor__colsample_bytree": uniform(
        loc=0.5, scale=0.5 
    ),
    'xgbregressor__gamma': [0.5, 1, 1.5, 2,3,4, 5],  
    'xgbregressor__subsample': [0.6,0.7, 0.8,0.9, 1.0],
}

In [49]:
#randomizedserchCV를 통해 하이퍼파라미터 탐색
from sklearn.model_selection import RandomizedSearchCV

randomized_search = RandomizedSearchCV(
    pipe,
    param_distributions=params,
    scoring="neg_mean_squared_error",
    n_iter=10,
    cv=3,
    verbose=3,
    random_state=42,
)
randomized_search.fit(X_train, y_train)

Fitting 3 folds for each of 10 candidates, totalling 30 fits


/Users/moon/opt/anaconda3/envs/ds_study/lib/python3.8/site-packages/xgboost/data.py:250: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):


[CV 1/3] END xgbregressor__colsample_bytree=0.6872700594236812, xgbregressor__gamma=3, xgbregressor__max_depth=6, xgbregressor__min_child_weight=5, xgbregressor__subsample=1.0;, score=-211366608.019 total time=   0.4s


/Users/moon/opt/anaconda3/envs/ds_study/lib/python3.8/site-packages/xgboost/data.py:250: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):


[CV 2/3] END xgbregressor__colsample_bytree=0.6872700594236812, xgbregressor__gamma=3, xgbregressor__max_depth=6, xgbregressor__min_child_weight=5, xgbregressor__subsample=1.0;, score=-253153951.489 total time=   0.3s


...

[CV 2/3] END xgbregressor__colsample_bytree=0.9711008778424264, xgbregressor__gamma=4, xgbregressor__max_depth=4, xgbregressor__min_child_weight=3, xgbregressor__subsample=0.6;, score=-306259852.861 total time=   0.3s


/Users/moon/opt/anaconda3/envs/ds_study/lib/python3.8/site-packages/xgboost/data.py:250: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):


[CV 3/3] END xgbregressor__colsample_bytree=0.9711008778424264, xgbregressor__gamma=4, xgbregressor__max_depth=4, xgbregressor__min_child_weight=3, xgbregressor__subsample=0.6;, score=-161641766.608 total time=   0.3s


/Users/moon/opt/anaconda3/envs/ds_study/lib/python3.8/site-packages/xgboost/data.py:250: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):


RandomizedSearchCV(cv=3,
                   estimator=Pipeline(steps=[('ordinalencoder',
                                              OrdinalEncoder()),
                                             ('xgbregressor',
                                              XGBRegressor(base_score=None,
                                                           booster=None,
                                                           colsample_bylevel=None,
                                                           colsample_bynode=None,
                                                           colsample_bytree=None,
                                                           enable_categorical=False,
                                                           eval_metric='rmse',
                                                           gamma=None,
                                                           gpu_id=None,
                                                           importance_type=None,


In [50]:
print("최적 하이퍼파라미터: ", randomized_search.best_params_)


최적 하이퍼파라미터:  {'xgbregressor__colsample_bytree': 0.5115312125207079, 'xgbregressor__gamma': 1.5, 'xgbregressor__max_depth': 6, 'xgbregressor__min_child_weight': 3, 'xgbregressor__subsample': 0.9}
최적 F1:  -193429815.72971883


In [116]:
#최적의 하이퍼파라미터 튜닝
pipe = make_pipeline(
    OrdinalEncoder(),
    XGBRegressor(
        objective="reg:squarederror",
        eval_metric="rmse",  
        n_estimators=200,
        random_state=42,
        n_jobs=-1,
        ###
        ###
        max_depth=6,
        learning_rate=0.1,
        min_child_weight=3,
        colsample_bytree=0.5115312125207079,
        gamma=1.5,
        subsample =0.9,
        
    ),
)
print(pipe)

        

Pipeline(steps=[('ordinalencoder', OrdinalEncoder()),
                ('xgbregressor',
                 XGBRegressor(base_score=None, booster=None,
                              colsample_bylevel=None, colsample_bynode=None,
                              colsample_bytree=0.5115312125207079,
                              enable_categorical=False, eval_metric='rmse',
                              gamma=1.5, gpu_id=None, importance_type=None,
                              interaction_constraints=None, learning_rate=0.1,
                              max_delta_step=None, max_depth=6,
                              min_child_weight=3, missing=nan,
                              monotone_constraints=None, n_estimators=200,
                              n_jobs=-1, num_parallel_tree=None, predictor=None,
                              random_state=42, reg_alpha=None, reg_lambda=None,
                              scale_pos_weight=None, subsample=0.9,
                              tree_method=None

In [117]:
pipe.fit(X_train, y_train)

/Users/moon/opt/anaconda3/envs/ds_study/lib/python3.8/site-packages/xgboost/data.py:250: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):


Pipeline(steps=[('ordinalencoder',
                 OrdinalEncoder(cols=['category'],
                                mapping=[{'col': 'category',
                                          'data_type': dtype('O'),
                                          'mapping': 기타 외식         1
치킨            2
한식            3
주점            4
제과제빵          5
분식            6
커피            7
패스트푸드         8
일식            9
피자           10
음료 (커피 외)    11
중식           12
기타 외국식       13
서양식          14
아이스크림/빙수     15
NaN          -2
dtype: int64}])),
                ('xgbregressor',
                 XGBRegressor(base_score=0.5, booster='gbtree',
                              colsample_bylevel=1, colsample_bynode=1,
                              colsample_bytree=0.5115312125...
                              gamma=1.5, gpu_id=-1, importance_type=None,
                              interaction_constraints='', learning_rate=0.1,
                              max_delta_step=0, max_depth=6, min_child_weight

In [7]:
y_pred_train = pipe.predict(X_train)
y_pred_val = pipe.predict(X_val)

In [8]:
# Calculate MAE

rmse_pred = mean_absolute_error(y_val, y_pred_val) 
print("Root Mean Absolute Error:" , np.sqrt(rmse_pred))


Root Mean Absolute Error: 77.15173924217659


In [121]:
#모델저장하기
joblib.dump(pipe, 'sales_predict_model.pkl') 


['sales_predict_model.pkl']